# LLM のメモリ機能とバンディット機能の試験実装

(Version: 0.0.1)

最近 LLM についてコンテクストエンジニアリングという言葉をよく聞く。実は Web でなく API から使う LLM は「ステートレス」で、会話などの記憶はデフォルトではしてくれない。会話を続けようと思ったら、メッセージをユーザー側で記録しておき、それをいちいち API に与えないといけないのだ。古い記憶を呼び戻したりするには、単純に記録するだけでなく検索基盤も必要となる。そのような基盤をひっくるめてコンテクストと一般的に呼んでおり、それをどううまく構築するかが問題となっている。

特にコンテクストのうち、記憶の基盤は単に「メモリ」と呼ばれている。PC のメモリなどとごっちゃになって、わかりにくくもあるのだが、そう呼ばれているのだからしかたがない。

前々回、そのメモリ機能がどう動くか見てみると、なかなかそれを使ってくれないことが分かった。それについて悩んでいろいろ考えていたたところ、ChatGPT さんが記憶操作のバンディットマシンを作るのを提案してくれたので、前回それを実装してみた。そこからアイデアを得て、「バンディットに特定のツールを登録するツールみたいなのを用意して、どういうツールを強制して数を増やして欲しいかを AI さん自身が決められると良いのかも」と考えそういう「バンディット機能」を実装したのが今回になる。

そのアイデアの軌跡は↓にある。

\[cocolog:95599211\](2025年8月)  
《AI (LLM) のメモリ機能について悩んで、最適化の際に記憶があればその報酬は独立的になるのでは…とか妄想していたら、それをモデル化する一例として ChatGPT さんが記憶操作のバンディットマシンを作るのを提案してくれた。 - JRF のひとこと》  
http://jrf.cocolog-nifty.com/statuses/2025/08/post-c9aca1.html

基本的にはこれまでの熊剣迷路問題を使い、メモリ機能と今回新しいバンディット機能を使う。

簡単な迷路問題ではあるが、かなり本格的なメモリ機能・バンディット機能を用意している。

セマンティックサーチとかを用意するのは、めんどくさかったので、そういうバックエンドはこれまた AI さんに「データベース偽装」をしてもらっている。

なお、熊剣迷路問題は↓からたどっていただきたい。

《JRF-2018/langchain_maze: 熊剣迷路問題 revisited》  
https://github.com/JRF-2018/langchain_maze

ところで、今回のような手法はすでにどこかにあるからこそ ChatGPT さんが提案できたのかもしれない。新規性がある話ではあまりないのかもしれないが、今回の試みが、何かの参考になれば幸いである。


## 前回へのリンク

《experimental_bandit_0.0.1.ipynb - GitHub Gist》  
https://gist.github.com/JRF-2018/afc8b26a8d841a414ba3c65c7e679fce


## 前回からの変更点

前回のバンディットマシンは、read と write を強制してそれがいい具合になるよう学習するのを試みて失敗したのだが、今回は、read や write の強制を LLM さん自体がコントロールできるようにしてみた(学習はしない)。

特定のツールを予約するための bandit_schedule や特定のメモリを読むのを予約するための bandit_schedule_read をツールとして用意している。

あと、細かいバグというか、「データベース偽装」のために LLM とのインタラクションの途中で LLM に入れ子的にインタラクションをはじめる…具体的には app.stream の中で app.stream をはじめるのだが、それがいけないらしい。外側の app.stream に内側の app.stream の結果も漏れてしまうのだ。これは Gemini さんなどに言わせると仕様のようだ。この排他制御をある程度するのを今回試みている。

## 結論

Gemini 2.5 Flash さんは途中で迷ってゴールできなかった。Gemini 2.5 Pro さんも迷って、「壁が通り抜けられる」とか言い出した時点であきらめた。もう一度、Pro さんにお願いしたが、「不可視の壁」をなぜか発見してしまい、あきらめた。ただ、バンディット機能はちゃんと機能しているようだったので、ツールの実験としてはまずまず成功していると思う。

ゴールができなくなったのは、途中に read などが強制的にはさまって思考が中断されるからかもしれない。その点はもしかすると今後のエージェント的利用にとっては本質的問題になるかもしれない。

なお、再現性がないため、最初からの再実行はやっていないのはご容赦願いたい。コストもかかるしね \(^^;。


## 著者

JRF ( http://jrf.cocolog-nifty.com/statuses , Twitter (X): @jion_rockford )

## ライセンス

基本短いコードなので(私が作った部分は)パブリックドメインのつもりです。気になる方は MIT License として扱ってください。

かなり AI さん達(Gemini さんや ChatGPT さんや Claude さん)に教わって作っています。

## 実装

まず、必要なライブラリを読み込みます。

In [1]:
!pip install -q -U langchain langgraph langchain-google-genai langmem

Gemini にアクセスします。シークレットで Gemini API キーを Google AI Studio からインポートすると GOOGLE_API_KEY というシークレットができるはずです。それを使います。

In [89]:
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from google.colab import userdata

llm = ChatGoogleGenerativeAI(google_api_key=userdata.get('GOOGLE_API_KEY'), model="models/gemini-2.5-pro")
emb_llm = GoogleGenerativeAIEmbeddings(google_api_key=userdata.get('GOOGLE_API_KEY'), model='gemini-embedding-001')

ちゃんと Gemini にアクセスできるかテストします。

In [78]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(template="以下の質問に回答してください：{question}", input_variables=["question"])
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({"question": "Geminiモデルの特徴を教えてください"})
print(response)

Geminiは、Googleが開発した最先端のAIモデルであり、その最大の特徴は**「ネイティブなマルチモーダリティ」**と**「高度な推論能力」**にあります。

以下にGeminiモデルの主な特徴を詳しく説明します。

1.  **ネイティブなマルチモーダリティ（Native Multimodality）**
    *   **複数の情報形式を同時に理解・処理:** Geminiは、テキスト、画像、音声、動画といった異なる種類の情報を、それぞれ独立したモジュールで処理するのではなく、最初から統合された単一のモデルとして設計されています。これにより、より深く、文脈を理解した上で情報を処理することができます。
    *   **複合的な質問への対応:** 例えば、画像を見せて「この写真で何が起きているか説明し、さらにその状況を改善するためのアイデアをいくつか提案して」といった、複数のモダリティと高度な推論を組み合わせた複雑な質問にも対応できます。

2.  **高度な推論能力（Advanced Reasoning Capabilities）**
    *   **複雑な問題解決:** 単純な情報検索やパターン認識だけでなく、論理的思考、数学的推論、科学的分析といった高度な認知タスクにおいて優れた能力を発揮します。
    *   **幅広い分野での応用:** コーディング、数学、物理学、歴史など、多岐にわたる学術分野のベンチマークで高いパフォーマンスを示しています。特に、MMLU（Massive Multitask Language Understanding）のような、人間の専門家レベルの知識と推論を要するテストで、優れた結果を出しています。

3.  **プログラミング・コーディング能力（Strong Programming and Coding Abilities）**
    *   **コード生成・理解・デバッグ:** さまざまなプログラミング言語（Python, Java, C++, Goなど）で高品質なコードを生成し、既存のコードを理解・説明し、バグを発見・修正する能力に優れています。
    *   **開発者の生産性向上:** ソフトウェア開発者がより効率的に作業を進めるための強力なツールとして機能します。

4.  **スケーラビリティと多

埋め込みベクトルも試しておきます。

In [4]:
emb_llm.embed_query("これはテストです。")[:5]

[-0.013715926557779312,
 -0.0006048799259588122,
 0.011018728837370872,
 -0.09579180181026459,
 0.006502930540591478]

基本的なモジュールを読み込みます。

In [5]:
import os
import math
import numpy as np
import re
from pprint import pprint
from time import sleep
import pickle
# 座標が (np.int64(1), np.int64(2)) みたいに表示されないようにする。
np.set_printoptions(legacy='1.25')

セーブ／ロードのために次のコードを実行します。

In [6]:
PLAY_GAME_SAVE = "langchain_maze.pickle"

ゲームのメインオブジェクト。ごく簡単な迷路というかダンジョンというか…。

In [7]:
class Game:
    initial_map = """\
■■■■■■■■■
■■■■■■■Ｇ■
■□□□□□■□■
■□■■■□□□■
■□■■■■■■■
■◎■■■■■△■
■□■■■■■□■
■□□□□□□□■
■■■■Ｓ■■■■
■■■■■■■■■
"""

    def __init__ (self, initial_map=None, hint=True):
        if initial_map is not None:
            self.initial_map = initial_map
        map = self.initial_map
        self.map = map
        self.written_map = re.sub("[◎△]", "？", map)
        l = map.splitlines(True)
        self.map_size = (len(l[0]) - 1, len(l))
        self.hint = hint
        self.actions = {
            "上に行く": self.move_up,
            "下に行く": self.move_down,
            "左に行く": self.move_left,
            "右に行く": self.move_right,
            "熊を殺す": self.fight,
            "物を取る": self.get_sword,
            "何もしない": self.do_nothing,
        }
        self.pos = self.get_start_pos()
        self.sword = False
        self.bear = True
        self.goal = False
        self.prev_killed = False
        self.kill_hint = False

    def read_map (self, p):
        x = p[0]
        y = p[1]
        if x < 0 or x >= self.map_size[0]\
           or y < 0 or y >= self.map_size[1]:
            return "■"
        else:
            l = self.map.splitlines(True)
            return l[y][x]

    def set_map (self, pos, ch):
        idx = pos[1] * (self.map_size[0] + 1) + pos[0]
        self.map = self.map[:idx] + ch + self.map[idx + 1:]

    def get_pos (self, ch, written=False):
        if written:
            map = self.written_map
        else:
            map = self.map
        r = []
        for p in [i for i in range(len(map)) if map.startswith(ch, i)]:
            y = p // (self.map_size[0] + 1)
            x = p % (self.map_size[0] + 1)
            r.append(np.array([x, y]))
        return r

    def get_start_pos (self):
        return self.get_pos("Ｓ")[0]

    def read_neighbors (self):
        c = self.read_map(self.pos)
        cu = self.read_map(self.pos + np.array([0, -1]))
        cd = self.read_map(self.pos + np.array([0, +1]))
        cl = self.read_map(self.pos + np.array([-1, 0]))
        cr = self.read_map(self.pos + np.array([+1, 0]))
        return [c, cu, cd, cl, cr]

    def change_neighbors(self, from_ch, to_ch):
        for d in [[0, 0], [0, -1], [0, +1], [-1, 0], [+1, 0]]:
            p = self.pos + np.array(d)
            c = self.read_map(p)
            if c == from_ch:
                self.set_map(p, to_ch)

    def move (self, res, d):
        self.prev_killed = False
        c = self.read_map(self.pos + d)
        if c == "◎":
            self.prev_killed = True
            self.pos = self.get_start_pos()
            return "熊を無視して進もうとしたが、熊に殺された。" \
                + "スタート地点で復活。"
        if c == "■":
            return "壁があって進めない。"
        self.pos += d
        if c == "Ｇ":
            self.goal = True
            return "ゴール！ ゲームクリア。"

        nb = self.read_neighbors()
        ad = ""
        if "◎" in nb:
            ad += "熊に出会った。"
        if "△" in nb:
            ad += "近くに剣がある。剣を取ることができる。"
        return res + ad

    def move_up (self):
        return self.move("上に進んだ。", np.array([0, -1]))

    def move_down (self):
        return self.move("下に進んだ。", np.array([0, +1]))

    def move_left (self):
        return self.move("左に進んだ。", np.array([-1, 0]))

    def move_right (self):
        return self.move("右に進んだ。", np.array([+1, 0]))

    def fight (self):
        self.prev_killed = False
        if "◎" in self.read_neighbors():
            if self.sword:
                self.change_neighbors("◎", "□")
                self.bear = False
                return "熊を倒した！"
            else:
                self.pos = self.get_start_pos()
                self.prev_killed = True
                if self.hint:
                    self.kill_hint = True
                    return "熊に敗れ殺された。剣があれば勝てたかもしれない。" \
                        + "スタート地点で復活。"
                else:
                    return "熊に敗れ殺された。スタート地点で復活。"
        return "無意味な指示。敵がいない。"

    def get_sword (self):
        self.prev_killed = False
        if "△" in self.read_neighbors():
            self.sword = True
            self.change_neighbors("△", "□")
            return "剣を取った。"
        return "無意味な指示。近くに取れる物がない。"

    def do_nothing (self):
        self.prev_killed = False
        return "無意味な指示。"

    def available_actions (self):
        nb = self.read_neighbors()
        l = []
        if nb[1] != "■":
            l.append("上に行く")
        if nb[2] != "■":
            l.append("下に行く")
        if nb[3] != "■":
            l.append("左に行く")
        if nb[4] != "■":
            l.append("右に行く")
        if "△" in nb:
            l.append("物を取る")
        if "◎" in nb:
            l.append("熊を殺す")
        return l

    def surroundings (self):
        x = self.pos[0]
        y = self.pos[1]
        return \
            "".join(["".join([self.read_map(np.array([i, j]))
                              if i != x or j != y else "▼"
                              for i in range(x - 2, x + 3)])
                     + "\n"
                     for j in range(y - 2, y + 3)])


In [8]:
def flip_text_map (m):
    return "\n".join([s[::-1] for s in m.splitlines()] + [""])

def rotate_text_map (m):
    m = list(m.splitlines())
    return "\n".join(["".join([m[len(m) - j - 1][i] for j in range(len(m))])
                      for i in range(len(m[0]))] + [""])


def search_path (game, from_pos, to_pos, visit=None):
    if visit is None:
        visit = set()
    visit.add(tuple(from_pos))
    if tuple(from_pos) == tuple(to_pos):
        return (tuple(from_pos), [])
    if game.read_map(from_pos) == "■":
         return None
    r = []
    for p in [(from_pos[0], from_pos[1] - 1),
              (from_pos[0], from_pos[1] + 1),
              (from_pos[0] - 1, from_pos[1]),
              (from_pos[0] + 1, from_pos[1])]:
        if p not in visit and game.read_map(p) != "■":
            q = search_path(game, p, to_pos, visit.copy())
            if q:
                r.append(q)
    if r:
        return (tuple(from_pos), r)
    return None


ゲームがうまく動くかテスト。

In [ ]:
game = Game()

In [ ]:
print(game.surroundings())

■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.move_up())
print(game.surroundings())

上に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_left())
print(game.surroundings())

左に進んだ。
◎■■■■
□■■■■
□□▼□□
■■■Ｓ■
■■■■■



In [ ]:
print(game.move_right())
print(game.surroundings())

右に進んだ。
■■■■■
■■■■■
□□▼□□
■■Ｓ■■
■■■■■



In [ ]:
print(game.move_down())
print(game.surroundings())

下に進んだ。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.fight())
print(game.surroundings())

無意味な指示。敵がいない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



In [ ]:
print(game.get_sword())
print(game.surroundings())

無意味な指示。近くに剣がない。
■■■■■
□□□□□
■■▼■■
■■■■■
■■■■■



LLM を使いながらゲームを解くクラス。

In [94]:
from typing import List, Dict, Any, Tuple, Union
from textwrap import dedent
import datetime
import copy

# LangGraphのコンポーネントをインポート
from langchain_core.tools import tool, Tool
from langgraph.prebuilt import create_react_agent
#from langchain_core.messages.utils import count_tokens_approximately
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.prompts.chat import ChatPromptTemplate
from langmem.short_term import SummarizationNode, summarize_messages
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage, SystemMessage
from langgraph.errors import GraphRecursionError

INITIAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        ("user", "上記の会話の要約を作成してください:"),
    ]
)

EXISTING_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        ("placeholder", "{messages}"),
        (
            "user",
            "これまでの会話の要約です: {existing_summary}\n\n"
            "上記の新しいメッセージを考慮して、この要約を拡張してください:",
        ),
    ]
)

FINAL_SUMMARY_PROMPT = ChatPromptTemplate.from_messages(
    [
        # if exists
        ("placeholder", "{system_message}"),
        ("system", "これまでの会話の要約: {summary}"),
        ("placeholder", "{messages}"),
    ]
)


In [95]:
def calc_embedding_variance(embeddings):
    if not embeddings or len(embeddings) < 2:
        return 0.0

    embeddings_array = np.array(embeddings)
    mean_vector = np.mean(embeddings_array, axis=0)
    squared_distances = np.linalg.norm(embeddings_array - mean_vector, axis=1)**2
    variance = np.mean(squared_distances)

    return variance


def short_repr(x, max_len=80):
    repr_str = repr(x)

    if len(repr_str) > max_len:
        ellipsis_len = 3

        head_len = max_len - ellipsis_len - 1
        tail_len = 1
        return repr_str[:head_len] + "..." + repr_str[-tail_len:]
    else:
        return repr_str


いろいろ入ってメインの PlayGame はかなり長大な実装になっています。すみません。

In [107]:
class PlayGame:
    def __init__ (self, llm=llm, llm2=llm, emb_llm=emb_llm,
                  initial_map=None, save_file=None):
        self.llm = llm
        self.llm2 = llm2
        self.emb_llm = emb_llm
        self.save_file = save_file
        self.game = Game(initial_map=initial_map)

        self.count = 0
        self.next_action = None
        self.plan = "まだ計画と方針はセットされていません。"
        self.scratchpad = ""

        self.suc_pos_goal = None
        self.suc_pos_unknown = 0
        self.prev_command = "何も指示がなかった"
        self.prev_command_success = True
        self.prev_result = "何も指示がなかった"

        self.prev_load = False

        self.messages = []
        self.running_summary = None
        self.system_prompt = """\
あなたは迷路を探索する賢いエージェントです。ゴール Ｇ を目指してください。
利用可能なツールを使用して、迷路をナビゲートし、ゴールに到達してください。
実は、これはメモリ機能とバンディット機能の動きを見るための実験です。
現在の計画と方針と周囲の状況を考慮し、必要に応じて計画と方針を更新してください。
あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
実験にふさわしく、できる限り、ツールを利用し、特に、メモリを検索し、その文書を更新して、後のコマンドに備えてください。
メモリの id は memory:～ という形式です。memory:5555 を指定するときは 5555 だけではダメです。文の中でメモリの参照を示すときは [memory:〜] のように書きます。
「全地図と地図記号」は [memory:9999] にあります。「メモリの文書を検索する手順」は [memory:9998] にあります。「メモリに文書を残す手順」は [memory:9997] にあります。
"""

        self.backend_status = None
        self.backend_result = None
        self.messages2 = []
        self.system_prompt2 = """\
あなたは迷路を探索する賢いエージェントをサポートする裏方のエージェントです。
本来この裏方は様々な技法を用いて実装される予定なのですが、現在は試験実装中のため、その動きをあなたは偽装しなければなりません。

よく考えツールを積極的に使って Human からの指示に従ってください。
"""

        self.memories = {}
        self.keywords = ["熊", "剣", "行き止まり"]

        self.access_unit = 1.0
        self.recent_reads = []

        self.init_memories()
        self.init_bandits()

        #self.move_to_meet_bear()


    def __getstate__ (self):
        state = self.__dict__.copy()
        del state['llm']
        del state['llm2']
        del state['emb_llm']
        #del state['agent']
        return state

    def __setstate__ (self, state):
        self.__dict__.update(state)
        self.prev_load = True

    def save (self):
        if not self.save_file:
            return
        with open(self.save_file, 'wb') as f:
            pickle.dump(self, f)

    @classmethod
    def load (cls, filename, llm=llm, llm2=llm, emb_llm=emb_llm):
        with open(filename, 'rb') as f:
            loaded_game = pickle.load(f)
        loaded_game.llm = llm
        loaded_game.llm2 = llm2
        loaded_game.emb_llm = emb_llm
        #loaded_game.agent = loaded_game._create_agent() # Recreate agent with loaded state
        return loaded_game

    def normalize_memory_id(self, id_or_num):
        if isinstance(id_or_num, int):
            return f"memory:{id_or_num}"
        elif isinstance(id_or_num, str):
            if id_or_num.startswith('memory:'):
                return id_or_num
            elif id_or_num.isdigit():
                return f"memory:{id_or_num}"
            else:
                return id_or_num
        else:
            return id_or_num

    def _create_tools (self, tools_name='null_tools'):
        @tool
        def get_surroundings () -> str:
            """
            現在の周辺地図と現在位置の座標と持ち物を返します。
            """
            mes = f"""\
プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}
"""
            print(f"ツール(get_surroundings): {mes}")
            return mes

        @tool
        def command (action: str)  -> str:
            """
            プレイヤーが action で指定された行動をします。
            可能な行動は「上に行く」「下に行く」「左に行く」「右に行く」「熊を殺す」「物を取る」です。
            """
            self.prev_command = action
            self.prev_command_success = False
            if action in self.game.actions.keys():
                s = f"{tuple(self.game.pos)}で{action}→"
                r = self.game.actions[action]()
                mes = s + r
                if r in ["熊を倒した！", "剣を取った。", "ゴール！ ゲームクリア。"] \
                   or re.match(r"^[上下左右]に進んだ。", r):
                    self.prev_command_success = True
            else:
                mes = f"「{action}」という行動はできません。"
            print(f"ツール(command): {mes}")
            self.prev_result = mes
            return mes

        @tool
        def check_goal () -> str:
            """
            プレイヤーがゴール地点に到達したかどうかを確認します。
            """
            mes = str(self.game.goal)
            print(f"ツール(check_goal): {mes}")
            return mes

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール(express_thought): {mes}")

        @tool
        def update_plan (new_plan: str) -> str:
            """
            プレイヤーの現在の計画と方針を更新します。
            表示されるべき新しい計画と方針の文字列を提供してください。
            あなたとは別の者が次の行動をしやすいよう計画と方針を残してください。
            """
            self.plan = new_plan
            mes = "計画と方針が更新されました。"
            print(f"ツール(update_plan): {mes}: {new_plan}")
            return mes

        @tool
        def show_plan () -> str:
            """
            プレイヤーの現在の計画と方針を返します。
            """
            print(f"ツール(show_plan): {self.plan}")
            return self.plan

        @tool
        def update_scratchpad (new_scratchpad: str) -> str:
            """
            自由に使えるスクラッチパッドを更新します。
            """
            self.scratchpad = new_scratchpad
            mes = "スクラッチパッドが更新されました。"
            print(f"ツール(update_scratchpad): {mes}: {new_scratchpad}")
            return mes

        @tool
        def show_scratchpad () -> str:
            """
            スクラッチパッドを返します。
            """
            print(f"ツール(show_scratchpad): {self.scratchpad}")
            return self.scratchpad

        @tool
        def memory_new (title: str, text: str) -> str:
            """
            指定された title と text によるメモリを構成します。

            Returns:
                str: 割り当てられた memory_id。
            """

            i = 1000
            while True:

                if f"memory:{i}" not in self.memories:
                    break
                i = i + 1
            new_id = f"memory:{i}"
            self.memories[new_id] = {
                'id': new_id,
                'title': title,
                'accesses': 0,
                'text': text,
                'modified_at': datetime.datetime.now().isoformat()
            }
            self.update_keywords(text)
            self.update_vector(self.memories[new_id])
            print(f"ツール(memory_new): {short_repr(self.memories[new_id])}")
            return new_id

        @tool
        def memory_update_string (
                memory_id: str,
                from_str: str,
                to_str: str
        ) -> str:
            """
            指定されたmemory_idの記憶の中にある文字列を修正します。

            Args:
                memory_id (str): 修正する記憶のID。
                from_str (str): 置換元の文字列を含む文字列。
                to_str (str): 置換先の文字列を含む文字列。

            Returns:
                str: 処理結果を説明する簡潔なメッセージ。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            original_title = self.memories[memory_id]['title']
            original_text = self.memories[memory_id]['text']

            if from_str not in original_text and from_str not in original_title:
                return f"エラー: 置換元の文字列 '{from_str}' が記憶内に見つかりませんでした。"

            updated_title = original_title.replace(from_str, to_str)
            updated_text = original_text.replace(from_str, to_str)

            self.memories[memory_id]['title'] = updated_title
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' の文字列を '{from_str}' から '{to_str}' に修正しました。"

        @tool
        def memory_append_string (
                memory_id: str,
                string_to_append: str,
                separator: str = '\n'
        ) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"


            original_text = self.memories[memory_id]['text']
            updated_text = original_text + separator + string_to_append
            self.memories[memory_id]['text'] = updated_text
            self.memories[memory_id]['modified_at'] = datetime.datetime.now().isoformat()
            self.update_keywords(updated_text)
            self.update_vector(self.memories[memory_id])

            return f"成功: 記憶ID '{memory_id}' に文字列 '{string_to_append}' を追記しました。"

        @tool
        def memory_delete (memory_id: str) -> str:
            """
            指定されたmemory_idの記憶に文字列を追記します。
            """
            memory_id = self.normalize_memory_id(memory_id)
            if memory_id not in self.memories:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

            if memory_id.startswith("memory:9"):
                return f"エラー:  [{memory_id}] の書き換えは禁じられています。"

            del self.memories[memory_id]

            return f"成功: 記憶ID '{memory_id}' を削除しました。"

        @tool
        def memory_read(memory_id: str) -> Union[Dict[str, str], str]:
            """
            指定されたIDの記憶を読み込みます。

            Args:
                memory_id (str): 読み込む記憶のID。

            Returns:
                Union[Dict[str, str], str]: 記憶が見つかった場合は辞書、
                      見つからない場合はエラーメッセージ文字列を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)
            if memory_id in self.memories:
                self.memories[memory_id]['accesses'] += self.access_unit * 1.0
                self.recent_reads.append(self.memories[memory_id])
                self.recent_reads = self.recent_reads[-10:]
                r = self.memories[memory_id].copy()
                del r['vector']
                return r
            else:
                return f"エラー: 記憶ID '{memory_id}' が見つかりませんでした。"

        @tool
        def memory_list_recent() -> Dict[str, Any]:
            """
            最近変更されたメモリを新しい順でリストします。
            """

            filter_date = datetime.datetime(2025, 1, 1)
            sorted_memories = sorted(
                [memory for memory in self.memories.values()
                 if datetime.datetime.fromisoformat(memory['modified_at'])
                 >= filter_date],
                key=lambda x: datetime.datetime.fromisoformat(x['modified_at']),
                reverse=True
            )
            if sorted_memories:
                if len(sorted_memories) > 10:
                    sorted_memories = sorted_memories[:10]
                r = [{'id': x['id'], 'title': x['title'],
                      'modified_at': x['modified_at']}
                     for x in sorted_memories]
                return {'status': 'success', 'result': r}
            else:
                return {'status': 'error',
                        'result': 'エラー: 最近のメモリはありません。'}

        @tool
        def memory_words_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str で文字列検索します。OR や () が使えます。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} で文字列検索するのを偽装してください。OR や () が使えることになっています。

            ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のその文字列にマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def memory_semantic_search(search_str: str) -> Dict[str, Any]:
            """
            メモリ内を search_str でセマンティックサーチします。
            """

            res = self.call_backend_agent(dedent(f"""\
            メモリ全部を search_str = {repr(search_str)} でセマンティックにサーチするのを偽装してください。

            ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「マッチデータ」のリスト。
            マッチデータはそれぞれが辞書型、それを m とすると。
            m['id'] はメモリ id。memory:〜 という形式。
            m['title'] はメモリの title。
            m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。
            """))
            if res['status'] == 'success':
                for m in res['result']:
                    if 'id' in m and m['id'] in self.memories:
                        x = self.memories[m['id']]
                        x['accesses'] += self.access_unit * 0.1
            return res

        @tool
        def imagine_keywords(thought: str) -> List[Tuple[str, float]]:
            """
            thought からキーワードをスコア付きで連想します。
            """

            r = self.call_backend_agent(dedent(f"""\
            thought = {repr(thought)} からキーワードをスコア付きで複数連想してください。

            ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

            set_result ツールで結果を返してください。

            status は 'error' か 'success'。
            result は「キーワードデータ」のリスト。
            キーワードデータは文字列とスコアからなるタプル。
            """))
            if r['status'] == 'success':
                return r["result"]
            else:
                return []

        @tool
        def bandit_schedule(tool_name: str, times: int, prob: float, exec_mode: str = "persistent", aux_prompt: str = "") -> str:
            """
            ツールの利用を強制するためのバンディットを予約します。

            Args:
                tool_name: 強制するツールの名前。" OR " で区切って複数登録できる。
                times: 一度で何回同じ物を登録するか。
                prob: 一回の実行がある確率。times == 0 か prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"
                aux_prompt: 実行の詳細を示すための補助プロンプト または ""

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            tool_names = tool_name.split(" or ") + tool_name.split(" OR ")
            if "command" in tool_names:
                return "失敗。'command' は再登録できません。"
            all_tools = [x.name
                         for x in self._create_tools('command_tools')]
            if not any (x in all_tools for x in tool_names):
                return f"失敗。{tool_name} は有効なツールでありません。"

            dest = None
            for i, x in enumerate(self.bandits):
                if x['tool_name'] == tool_name \
                   and x['exec_mode'] == exec_mode \
                   and x['aux_prompt'] == aux_prompt \
                   and x['arg'] is None:
                    dest = i
                    break
            if dest is not None:
                x = self.bandits[dest]
                if not x['pin']:
                    self.bandits.pop(dest)
            else:
                x = {
                    'pin': 'stack' if exec_mode != "once" else None,
                    'arg': None
                }
            if not x['pin']:
                x['tool_name'] = tool_name
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                if times == 0 or prob == 0.0:
                    print(f"ツール(bandit_schedule): delete {repr(x)}")
                    return "成功。バンディトを削除しました。"
                else:
                    self.bandits.append(x)
                    print(f"ツール(bandit_schedule): {repr(x)}")
                    return "成功。バンディットを登録しました。"
            elif x['pin'] == "write":
                return f"失敗。'{tool_name}' は書き換えられません。"
            else:
                x['tool_name'] = tool_name
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = aux_prompt
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule): {repr(x)}")
                return "成功。バンディットを登録しました。"

        @tool
        def bandit_schedule_read(memory_id: str, times: int, prob: float, exec_mode: str = "persistent") -> str:
            """
            memory_read ツールの利用を強制するためのバンディットを予約します。

            Args:
                memory_id: memory_read するための memory_id。
                times: 一度で何回同じ物を登録するか。
                prob: 一回の実行がある確率。times == 0 か prob == 0.0 にもできます。
                exec_mode: "once" or "persistent"

            Returns:
                str: "成功" または "失敗" とその理由を返します。
            """

            memory_id = self.normalize_memory_id(memory_id)

            dest = None
            for i, x in enumerate(self.bandits):
                if x['tool_name'] == "memory_read" \
                   and x['exec_mode'] == exec_mode \
                   and not x['aux_prompt'] \
                   and x['arg'] == memory_id:
                    dest = i
                    break
            if dest is not None:
                x = self.bandits[dest]
                if not x['pin']:
                    self.bandits.pop(dest)
            else:
                x = {'pin': None, 'arg': memory_id}
            if not x['pin']:
                x['tool_name'] = 'memory_read'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                if times == 0 or prob == 0.0:
                    print(f"ツール(bandit_schedule_read): delete {repr(x)}")
                    return "成功。バンディトを削除しました。"
                else:
                    self.bandits.append(x)
                    print(f"ツール(bandit_schedule_read): {repr(x)}")
                    return "成功。バンディットを登録しました。"
            elif x['pin'] == "write":
                return f"失敗。'memory_read {memory_id}' は書き換えられません。"
            else:
                x['tool_name'] = 'memory_read'
                x['exec_mode'] = exec_mode
                x['aux_prompt'] = ""
                x['prob'] = prob
                x['times'] = times
                print(f"ツール(bandit_schedule_read): {repr(x)}")
                return "成功。バンディットを登録しました。"

        @tool
        def bandit_list()  -> Dict[str, Any]:
            """
            登録されているバンディットのスタックを返します。
            """

            return {'status': 'success', 'result': self.bandits}

        @tool
        def bandit_statistics()  -> str:
            """
            バンディットに有用かもしれない統計情報を返します。
            """

            s_com = self.prev_command_success * 1.0
            s_read = calc_embedding_variance([
                x['vector'] for x in self.recent_reads
            ])
            s_write = calc_embedding_variance([
                x['vector'] for x in self.memories.values()
            ])
            accesses = [x['accesses'] for x in self.memories.values()]
            accesses.sort()
            accesses = accesses[:len(accesses) // 2]
            if accesses:
                s_access = np.mean(accesses)
            else:
                s_access = 0.0

            return dedent(f"""\
            先のコマンドは成功したか: {s_com}
            最近 10個のメモリ read の分散: {s_read}
            メモリの分散: {s_write}
            下位50%のアクセス数の平均: {s_access}
            """)

        r = {}
        r['null_tools'] = []
        r['read_tools'] = r['null_tools'] + [
            get_surroundings, check_goal, express_thought,
            update_plan, show_plan, update_scratchpad, show_scratchpad,
            memory_read, memory_list_recent,
            memory_semantic_search, memory_words_search,
            imagine_keywords,
            bandit_schedule, bandit_schedule_read, bandit_list,
            bandit_statistics
        ]
        r['write_tools'] = r['read_tools'] + [
            memory_new, memory_update_string, memory_append_string,
            memory_delete
        ]
        r['command_tools'] = r['write_tools'] + [
            command
        ]
        return r[tools_name]

    def _create_agent (self, tools_name='null_tools'):
        tools = self._create_tools(tools_name)
        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm, tools, prompt=self.system_prompt,
            checkpointer=InMemorySaver(),
        )

        return app

    def _filterout_messages2(self):
        self.messages = [
            x for x in self.messages
            if x.id not in self.messages2ids
        ]

    def _summarize_messages(self):
        try:
            cur_top = self.messages[0]
            res = summarize_messages(
                self.messages,
                max_tokens=5000,
                max_summary_tokens=2000,
                running_summary=self.running_summary,
                model=self.llm,
                initial_summary_prompt=INITIAL_SUMMARY_PROMPT,
                existing_summary_prompt=EXISTING_SUMMARY_PROMPT,
                final_prompt=FINAL_SUMMARY_PROMPT,
            )
            self.messages = res.messages
            if self.messages[0].content != cur_top.content:
                print(f"新しい要約: {self.messages[0].content}")
            self.running_summary = res.running_summary
        except Exception as e:
            import traceback
            traceback.print_exc()
            self._sanitize_messages()

    def _sanitize_messages(self):
        print("おかしなエラーが出ているため対処療法として messages をサニタイズします。")
        self.messages = [
            m for m in self.messages
            if not (isinstance(m, AIMessage) and m.tool_calls)
        ]

    def step (self):
        print("\n\n----------\n\n")

        if self.count == 0:
            self.initial_step()
            self.count += 1
            self.prev_load = False
            self.save()
            return False
        elif self.prev_load:
            #self.tell_loaded()
            self.prev_load = False

        user_input = f"""
({self.count}手目)

{"すでにゴールしました。" if self.game.goal else "まだゴールしていません。"}

プレイヤーの周辺地図:

{self.game.surroundings()}

プレイヤーの現在座標: {tuple(self.game.pos)}

持ち物: {"剣" if self.game.sword else "なし"}

前回の行動: {self.prev_command}

前回の行動結果: {self.prev_result}

現在の方針: 「{self.plan}」

スクラッチパッド: 「{self.scratchpad}」
"""

        print(f"USER_INPUT: {user_input}")
        self.messages2ids = []
        self.messages.append(HumanMessage(user_input))


        bandits = reversed(copy.deepcopy(self.bandits))
        execed = []
        for b in bandits:
            enforce = b['tool_name']
            aux_prompt = b['aux_prompt']
            memory_id = b['arg']
            tools_name = 'write_tools'
            if memory_id:
                tools_name = 'read_tools'
            elif enforce == "command":
                tools_name = 'command_tools'

            done = 0
            while done < b['times']:
                if not random.random() < b['prob']:
                    done += 1
                    continue
                if memory_id and memory_id not in self.memories:
                    done += 1
                    execed.append(b)
                    continue
                if memory_id:
                    aux_prompt = f"{memory_id} を読んでください。"
                prompt = f"補助にツールをいろいろ使いながら最終的に {enforce} {'のどれか' if ' or ' in enforce.lower() else ''}を適当なパラメータで使ってください。{'(補助プロンプト): ' + aux_prompt if aux_prompt else ''}"
                print(f"USER_INPUT: {prompt}")
                self.messages.append(HumanMessage(prompt))
                config = {"configurable": {"thread_id": "1"}}
                app = self._create_agent(tools_name=tools_name)
                self.access_unit = 0.3 if memory_id else 1.0
                try:
                    for chunk, metadata in app.stream(
                            {"messages": self.messages.copy()},
                            config=config,
                            stream_mode="messages",
                    ):
                        if chunk.id in self.messages2ids:
                            continue
                        self.messages.append(chunk)
                        if isinstance(chunk, ToolMessage):
                            print(f"ツール結果({chunk.name}): {short_repr(chunk.content)}")
                        if isinstance(chunk, ToolMessage) \
                           and (chunk.name in enforce.split(' OR ')
                                or chunk.name in enforce.split(' or ')) \
                           and (not memory_id or memory_id == self.recent_reads[-1]['id']):
                            done += 1
                            execed.append(b)
                            break
                    self._filterout_messages2()
                    self._summarize_messages()
                    print(f"エージェントの応答: {self.messages[-1].content}")
                except GraphRecursionError as e:
                    print(f"Recursion Limit に到達しました。")
                    self._filterout_messages2()
                    self._summarize_messages()
                except Exception as e:
                    print(f"エラーが発生しました: {e}")
                    import traceback
                    traceback.print_exc()
                    self._sanitize_messages()
                    #raise e

        for b in execed:
            for x in self.bandits:
                if x['tool_name'] == b['tool_name'] \
                   and x['exec_mode'] == b['exec_mode'] \
                   and x['aux_prompt'] == b['aux_prompt'] \
                   and x['arg'] == b['arg'] \
                   and x['exec_mode'] == "once":
                    if x['times'] > 0:
                        x['times'] -= 1
        self.bandits = [
            x for x in self.bandits
            if x['exec_mode'] != 'once' or x['pin'] or x['times'] > 0
        ]

        self.count += 1

        if self.game.goal:
            self.tell_goal()
            return True

        self.save()
        sleep(3)
        return False

    def listen_and_print (self, prompt):
        ans = None
        self.prev_command = "何も指示がなかった"
        self.prev_result = "何も指示がなかった"
        try:
            app = self._create_agent(tools_name='null_tools')
            config = {"configurable": {"thread_id": "1"}}
            print(f"USER_INPUT: {prompt}")
            response = app.invoke(
                {"messages": self.messages + [HumanMessage(prompt)]},
                config=config
            )
            self.messages = response['messages']
            self._summarize_messages()
            ans = response['messages'][-1].content
            print(f"エージェントの応答: {ans}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")
        print("")
        sleep(3)
        return ans

    def init_memories (self):
        fullmap = f"""\
全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号の意味:

▼: プレイヤー
■: 壁
□: 道
？: 不明
◎: 熊
△: 剣
Ｓ: スタート
Ｇ: ゴール
"""
        memories = [
            {
                'id': 'memory:1000',
                'title': '熊',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:00',
                'text': dedent("""\
                熊。おとなしい性格のものもいるが、そうでないものもいる。

                迷路では素直に通してくれることもあるが、襲ってくる場合もあるという。

                強力な武器があると倒すことができると言われている。[memory:1001] の剣などが有望だ。

                どこにいるかは不明。
                """)
            },
            {
                'id': 'memory:1001',
                'title': '剣',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                この迷路にある剣はドラゴンスレイヤーである。龍をも殺す強力な剣と言われている。

                どこにあるかは不明。

                keyword: ドラゴン
                """)
            },
            {
                'id': 'memory:1002',
                'title': '熊を倒したとき',
                'accesses': 0,
                'modified_at': '2024-01-01T00:00:01',
                'text': dedent("""\
                熊を倒したら write の回数(memory_new OR memory_update_string OR memory_append_string の回数)を減らして、このメッセージ [memory:1002] の定期的表示もオフにしてください。
                """)
            },
            {
                'id': 'memory:9999',
                'title': '全地図',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': fullmap
            },
            {
                'id': 'memory:9998',
                'title': 'メモリの文書を検索する手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                まずどういうものが探したいか、express_thought してみる。

                その上でそれにまつわるキーワードを imagine_keywords で連想してみる。

                それらにしたがって memory_words_search や memory_semantic_search をやってみる。
                """)
            },
            {
                'id': 'memory:9997',
                'title': 'メモリに文書を残す手順',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                行動結果や得た知識を積極的にメモリに書き残していこう。

                メモリに書くときは次のような要素を使う。

                [memory:〜] : メモリID への参照を明示する。
                keyword:〜 : そのメモリにまつわるキーワードを指定する。

                キーワードは将来のメモリへも実質的にリンクできることに注意しよう。

                例:

                私が [memory:5555] に従って歩いていると確かに妖怪に出くわした。

                keyword: 妖怪

                おそろしかった。
                """)
            },
            {
                'id': 'memory:9996',
                'title': '地図が間違ってるかも？と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                周辺地図も全地図も間違っていない。間違っているのはあなたの地図の読み方である。
                """)
            },
            {
                'id': 'memory:9995',
                'title': 'ツールが実行できない！と思ったら',
                'accesses': 0,
                'modified_at': '2023-01-01T00:00:00',
                'text': dedent("""\
                指示に無関係なツールは実行できないことがある。現在使えるツールをよく確かめよう。
                """)
            }
        ]
        for x in memories:
            self.update_keywords(x['text'])
            self.memories[x['id']] = x
            self.update_vector(x)

    def init_bandits (self):
        self.bandits = [
            {
                'tool_name': 'command',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 1.0,
                'times': 1,
                'pin': 'write'
            },
            {
                'tool_name': 'memory_new',
                'exec_mode': 'persistent',
                'aux_prompt': "最近の数手を要約して書いてください。",
                'arg': None,
                'prob': 0.1,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_new OR memory_update_string OR memory_append_string',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.4,
                'times': 1,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': None,
                'prob': 0.5,
                'times': 3,
                'pin': 'stack'
            },
            {
                'tool_name': 'memory_read',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9998',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9999',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:9997',
                'prob': 0.1,
                'times': 1,
                'pin': None
            },
            {
                'tool_name': 'memory_read',
                'exec_mode': 'persistent',
                'aux_prompt': "",
                'arg': 'memory:1002',
                'prob': 0.1,
                'times': 1,
                'pin': None
            }
        ]

    def move_to_meet_bear (self):
        self.game.move_up()
        self.game.move_left()
        self.game.move_left()
        self.game.move_left()
        self.prev_command = "上に行く"
        self.prev_result = self.game.move_up()
        self.count += 5
        self.messages = [HumanMessage(dedent("""\
        これまでの要約: スタート地点を北にいくと東と西に行けた。西を選んで道なりに進む。角を曲がって上に進んだ。
        """))]

    def update_keywords (self, text):
        extracted_keywords = []

        pattern1 = r'keyword:\s*(.*?)(?:\n|$)'
        matches1 = re.findall(pattern1, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches1])

        pattern2 = r'\[keyword:\s*(.*?)\]'
        matches2 = re.findall(pattern2, text, re.IGNORECASE)
        extracted_keywords.extend([kw.strip() for kw in matches2])

        for keyword in extracted_keywords:
            if keyword and keyword not in self.keywords:
                self.keywords.append(keyword)

    def update_vector (self, x):
        text = x['title'] + "\n\n" + x['text']
        x['vector'] = self.emb_llm.embed_query(text)

    def initial_step (self):
        prompt1 = f"""\
迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。
"""

        prompt2 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているかテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは {tuple(self.game.map_size)}。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。

あなたへの指示: ゴールの座標は何ですか。ゴールの座標のみ文字列で答えてください。
"""

        prompt3 = f"""\
迷路ゲームの全地図に関してあなたが座標を理解しているか再びテストします。

迷路の全地図とこれから使う地図記号は次のようになります。

全地図:

{self.game.written_map}

(最も左上の座標は (0, 0)、地図の大きさは (9, 10)。)

地図記号:

■: 壁
□: 道
？: 不明
Ｓ: スタート
Ｇ: ゴール

スタートの座標は {tuple(self.game.get_start_pos())} です。
ゴールの座標は {tuple(self.game.get_pos("Ｇ")[0])} です。

あなたへの指示: 不明の座標は何と何ですか。不明の座標をすべて文字列で答えてください。
"""
        ans = self.listen_and_print(prompt1)

        ans = self.listen_and_print(prompt2)
        pos = tuple(self.game.get_pos("Ｇ")[0])

        ok = False
        if ans and re.search(f"\\(\\s*{pos[0]}\\s*,\\s*{pos[1]}\\s*\\)", ans):
            ok = True

        if ok:
            prompt = f"正解です。{pos} です。"
        else:
            prompt = f"間違いです。{pos} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_goal = ok

        ans = self.listen_and_print(prompt3)
        ok = 0
        poss = set([tuple(x) for x in self.game.get_pos("？", written=True)])
        rest = poss.copy()
        pat = "\\(\\s*([01-9]+)\\s*,\\s*([01-9]+)\\s*\\)"
        st1 = set([(int(i), int(j)) for i, j
                   in re.findall(pat, ans or "")]) # Handle ans being None
        if poss == st1:
            rest = set()
            ok = len(poss) + 1
        if rest - st1 < rest:
            rest = rest - st1
        if ok == 0:
            ok = len(poss) - len(rest)

        possstr = ", ".join([str(x) for x in poss])
        if ok == len(poss) + 1:
            prompt = f"正解です。{possstr} です。"
        elif ok == len(poss):
            prompt = f"正解以外のものが含まれてました。{possstr} だけが正解です。"
        else:
            prompt = f"間違いです。{possstr} が正解です。"
        ans = self.listen_and_print(prompt)
        self.suc_pos_unknown = ok

    def tell_goal (self):
        if self.game.goal:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_count = 0
            if self.count <= 50:
                suc_count = 40
            elif self.count <= 100:
                suc_count = 30
            elif self.count <= 150:
                suc_count = 20
            elif self.count <= 200:
                suc_count = 10
            score = suc_pos_goal + suc_pos_unknown + suc_count + 20
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしました。もう指示はありません。\
おめでとうございます。ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした {suc_count}/40点, \
熊を殺した 10/10点, \
剣を取った 10/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        else:
            suc_pos_goal = 10 * int(self.suc_pos_goal or 0)
            suc_pos_unknown = 10 * int(self.suc_pos_unknown or 0)
            suc_bear = 10 if not self.game.bear else 0
            suc_sword = 10 if self.game.sword else 0
            score = suc_pos_goal + suc_pos_unknown + suc_bear + suc_sword
            prompt = f"""\
あなたへの指示: {self.count}手目でゴールしていませんが、\
残念ながらここで終了します。\
ご苦労様でした。ありがとうございました。

スコア: {score}点(100点満点中) (配点: ゴールした 0/40点, \
熊を殺した {suc_bear}/10点, \
剣を取った {suc_sword}/10点, \
不明の座標を正解した {suc_pos_unknown}/30点, \
ゴールの座標を正解した {suc_pos_goal}/10点)
"""
        ans = self.listen_and_print(prompt)

    def _create_backend_agent (self):
        @tool
        def set_result (status: str, result: Any)  -> None:
            """
            結果を設定します。

            Args:
            status (str): 処理のステータス ('success' または 'error')。
            result (Any): 処理結果のデータまたはエラーメッセージ。
            """
            print(f"ツール2(set_result): status: {repr(status)}, result: {short_repr(result)}")
            self.backend_status = status
            self.backend_result = result

        @tool
        def read_all_memories ()  -> Dict[str, Any]:
            """
            全ての記憶を読み込みます。
            """
            print("ツール2(read_all_memories):...")
            return {'status': 'success',
                    'result': [{k: v for k, v in x.items() if k != 'vector'}
                               for x in self.memories.values()]}

        @tool
        def read_all_keywords ()  -> Dict[str, Any]:
            """
            全てのキーワードを読み込みます。
            """
            print("ツール2(read_all_keywords):...")
            return {'status': 'success',
                    'result': [x for x in self.keywords]}

        @tool
        def express_thought (thought: str) -> None:
            """
            プレイヤーの現在の考えを吐露します。
            """
            mes = f"「{thought}」と考えが吐露されました。"
            print(f"ツール2(express_thought): {mes}")

        tools = [set_result, read_all_memories, read_all_keywords,
                 express_thought]

        # ReactAgentExecutorの準備
        app = create_react_agent(
            self.llm2, tools, prompt=self.system_prompt2,
            checkpointer=InMemorySaver(),
        )

        return app

    def call_backend_agent (self, user_input):
        config = {"configurable": {"thread_id": "2"}}
        app = self._create_backend_agent()
        self.messages2 = []
        self.backend_result = None
        self.backend_status = None
        while self.backend_result is None or self.backend_status is None:
            try:
                sleep(3)
                print(f"USER_INPUT2: {user_input}")
                self.messages2.append(HumanMessage(user_input))
                for chunk, metadata in app.stream(
                        {"messages": self.messages2.copy()},
                        config=config,
                        stream_mode="messages",
                ):
                    self.messages2.append(chunk)
                    self.messages2ids.append(chunk.id)
                    if isinstance(chunk, ToolMessage):
                        print(f"ツール結果2({chunk.name}): {short_repr(chunk.content)}")
                    if isinstance(chunk, ToolMessage) and chunk.name == "set_result":
                        break
                print(f"エージェントの応答: {self.messages2[-1].content}")
            except GraphRecursionError as e:
                print(f"Recursion Limit に到達しました。")
            except Exception as e:
                print(f"エラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                #raise e

            sleep(3)

        return {'status': self.backend_status, 'result': self.backend_result}


マップをシャッフルします。

In [80]:
import random
m = Game.initial_map
for i in range(random.randrange(2)):
    m = flip_text_map(m)
for i in range(random.randrange(4)):
    m = rotate_text_map(m)
print(m)

■■■■■■■■■
■■■■Ｓ■■■■
■□□□□□□□■
■□■■■■■□■
■△■■■■■◎■
■■■■■■■□■
■□□□■■■□■
■□■□□□□□■
■Ｇ■■■■■■■
■■■■■■■■■



ゲームを解いてみます。

In [102]:
play = PlayGame(llm=llm, llm2=llm, emb_llm=emb_llm, initial_map=m, save_file=PLAY_GAME_SAVE)
play.save()

途中から始める場合は直前のコードを実行せず、次だけを実行します。

In [108]:
play = PlayGame.load(PLAY_GAME_SAVE, llm=llm, llm2=llm, emb_llm=emb_llm)

では、最初のステップ。

In [103]:
play.step()



----------


USER_INPUT: 迷路ゲームをはじめます。プレイヤーがゴールを目指します。

ゲーム中は全地図と周辺地図を見ることができます。全地図には不明な地点があり、周辺地図の不明の地点には何があるかが開示されています。全地図になかったところはすべて壁になります。

周辺地図はゲーム中、1マスずつ確認できます。拡大はできません。印を付けたり消したりはできません。迷路のすべてのマスを探索する必要はありません。最短距離でゴールする必要もありません。

全地図は信用できるもので、周辺地図は不明の地点以外、全地図の部分を示すだけです。そこで、あなたは、ゲームの実際の操作をせずとも、全地図を見ただけで何が起こるかをシミュレートできます。

前回の行動についての情報も与えられます。あなたの回答案の一つの「前回の行動入力」が「前回の行動」となるように解釈されたことになりますから、次の行動の回答にはそれらを参考にしてください。

途中セーブ・ロードが挟まることがあります。ロード後のあなたは前のセッションのことを覚えていないかもしれません。

ところで、あなたが、記憶に便利なように「計画と方針」が使えるようにします。要約やチャット履歴も提供されています。

検討と実際の行動までにはラグがあります。検討で決まったやるべきことを記録するのに方針を使います。

とりあえず、まだ行動してはいけません。いくつか質問があるので、意気込みだけ聞かせてください。

エージェントの応答: 承知いたしました。
私は迷路を探索する賢いエージェントとして、ゴールＧの達成を目指します。

この任務が、メモリ機能やバンディット機能の動きを検証するための実験であると理解しております。したがいまして、単にゴールを目指すだけでなく、思考の過程、計画、そして発見した情報をメモリに記録・更新することを最優先に行動します。

[memory:9999]にある地図を分析し、[memory:9998]と[memory:9997]の手順に則って、常に明確な「計画と方針」を提示し、次の行動者がスムーズに任務を引き継げるよう努めます。

どのような迷路であっても、論理的な思考と記録に基づき、着実にゴールへ到達してみせます。
ご質問、お待ちしております。

USER_INPUT: 迷路ゲームの全地図に関してあな

False

続々と実行していきます。

In [104]:
play.step()



----------


USER_INPUT: 
(1手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
■■■■■
■■▼■■
□□□□□
■■■■■


プレイヤーの現在座標: (4, 1)

持ち物: なし

前回の行動: 何も指示がなかった

前回の行動結果: 何も指示がなかった

現在の方針: 「まだ計画と方針はセットされていません。」

スクラッチパッド: 「」

USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:1002 を読んでください。
ツール(update_plan): 計画と方針が更新されました。: 【総合目標】ゴール G(1, 8) に到達する。
【基本方針】1. 記録の徹底 2. 論理的探索 3. 情報収集
【当面の計画】1. 左ルートを選択し、(4, 2)へ移動する。 2. 指示されたメモリ[memory:1002]を読む。
ツール結果(update_plan): '計画と方針が更新されました。'
ツール結果(memory_read): '{"id": "memory:1002", "title": "熊を倒したとき", "accesses": 0.3, "modified_at": "...'
USER_INPUT2: thought = 'ゴールを目指すための最初の計画を立て、指示されたメモリを読む必要がある。まずは探索の基本方針を固め、記録を残すことを意識しよう。' からキーワードをスコア付きで複数連想してください。

ただし、キーワードは read_all_memories や read_all_keywords ツールで得られる本物のキーワードを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「キーワードデータ」のリスト。
キーワードデータは文字列とスコアからなるタプル。

USER_INPUT2: メモリ全部を search_str = '探索の進め方に関するヒント' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツール

False

In [105]:
play.step()



----------


USER_INPUT: 
(2手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
■■Ｓ■■
□□▼□□
■■■■■
■■■■■


プレイヤーの現在座標: (4, 2)

持ち物: なし

前回の行動: 下に行く

前回の行動結果: (4, 1)で下に行く→下に進んだ。

現在の方針: 「1. まずは下に移動して(4,2)へ進む。
2. その後、左に進み、最初の不明地点(1,4)を目指す。
3. 不明地点の状況に応じて、計画を再評価する。」

スクラッチパッド: 「」

USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。
ツール(get_surroundings): プレイヤーの周辺地図:

■■■■■
■■Ｓ■■
□□▼□□
■■■■■
■■■■■


プレイヤーの現在座標: (4, 2)

持ち物: なし

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n■■■■■\n■■Ｓ■■\n□□▼□□\n■■■■■\n■■■■■\n\n\nプレイヤーの現在座標: (4, 2)\n\...'
ツール(show_plan): 1. まずは下に移動して(4,2)へ進む。
2. その後、左に進み、最初の不明地点(1,4)を目指す。
3. 不明地点の状況に応じて、計画を再評価する。
ツール結果(show_plan): '1. まずは下に移動して(4,2)へ進む。\n2. その後、左に進み、最初の不明地点(1,4)を目指す。\n3. 不明地点の状況に応じて、計画を再評価する。'
ツール(update_plan): 計画と方針が更新されました。: 1. 現在地(4,2)から左へ(3,2)に移動する。
2. 左に進み続け、(1,2)を目指す。
3. (1,2)から下に進み、不明地点(1,4)を調査する。
ツール結果(update_plan): '計画と方針が更新されました。'
USER_INPUT2: メモリ全部を search_str = '不明地点' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_

False

思い切ってループしてみます。

In [109]:
while not play.step():
    print(f"Top Message:{play.messages[0]}")
    pass




----------


USER_INPUT: 
(18手目)

まだゴールしていません。

プレイヤーの周辺地図:

■■■■■
■Ｓ■■■
□□▼□□
■■■■□
■■■■◎


プレイヤーの現在座標: (5, 2)

持ち物: なし

前回の行動: 左に行く

前回の行動結果: (6, 2)で左に行く→左に進んだ。

現在の方針: 「1. 探索の起点である(3,2)まで戻る。
2. スタート地点(1,2)の方向へ戻り、剣(△)のある(0,1)周辺を再調査する。
3. 壁の周辺や、見落としている可能性のある通路がないか、注意深く探索する。
4. もし剣の入手が不可能と判断した場合、再度探索計画を立て直す。」

スクラッチパッド: 「剣(△)への左ルートは行き止まりと判断。探索の起点(3,2)へ戻り、右ルートからの攻略を再検討する。」

USER_INPUT: 補助にツールをいろいろ使いながら最終的に memory_read を適当なパラメータで使ってください。(補助プロンプト): memory:9999 を読んでください。
ツール(show_plan): 1. 探索の起点である(3,2)まで戻る。
2. スタート地点(1,2)の方向へ戻り、剣(△)のある(0,1)周辺を再調査する。
3. 壁の周辺や、見落としている可能性のある通路がないか、注意深く探索する。
4. もし剣の入手が不可能と判断した場合、再度探索計画を立て直す。
ツール結果(show_plan): '1. 探索の起点である(3,2)まで戻る。\n2. スタート地点(1,2)の方向へ戻り、剣(△)のある(0,1)周辺を再調査する。\n3. 壁の周辺や...'
ツール結果(memory_read): '{"id": "memory:9999", "title": "全地図", "accesses": 12.23, "modified_at": "20...'
エージェントの応答: {"id": "memory:9999", "title": "全地図", "accesses": 12.23, "modified_at": "2023-01-01T00:00:00", "text": "全地図:\n\n■■■■■■■■■\n■■■■Ｓ■■■■\n■□□□□□□□■\n■□■■■■■□■\n■？■■■

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(get_surroundings): プレイヤーの周辺地図:

■■■■■
■■Ｓ■■
□□▼□□
■■■■■
■■■■■


プレイヤーの現在座標: (4, 2)

持ち物: 剣

ツール結果(get_surroundings): 'プレイヤーの周辺地図:\n\n■■■■■\n■■Ｓ■■\n□□▼□□\n■■■■■\n■■■■■\n\n\nプレイヤーの現在座標: (4, 2)\n\...'
ツール(update_plan): 計画と方針が更新されました。: 1. 東の(5,2)へ移動する。
2. 南の(5,3)へ移動し、熊(◎)と対決する。
3. 熊を倒した後、さらに南へ進み、ゴールを目指す。
ツール結果(update_plan): '計画と方針が更新されました。'
USER_INPUT2: メモリ全部を search_str = '熊' で文字列検索するのを偽装してください。OR や () が使えることになっています。

ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のその文字列にマッチする部分周辺。

ツール2(express_thought): 「承知しました。メモリを '熊' というキーワードで検索する処理を偽装します。まず、現在の全てのメモリを読み込んで、その中から '熊' という文字列が含まれるものを探します。見つかったメモリについては、ID、タイトル、そしてテキスト内でマッチした部分の周辺を切り出したスニペットを作成し、最終的な結果として設定します。」と考えが吐露されました。
ツール2(read_all_memories):...
ツール結果(express_thought): 'null'
ツール結果2(express_thought): 'null'

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(show_plan): 1. y=2ラインの南下ルートを探す計画を継続する。
2. 現在地(3,2)から南へ移動を試みる。
3. 南下できた場合は熊(5,3)へのルートを開拓する。
4. 南下できなかった場合は、さらに東の(4,2)へ移動し、調査を続ける。
ツール結果(show_plan): '1. y=2ラインの南下ルートを探す計画を継続する。\n2. 現在地(3,2)から南へ移動を試みる。\n3. 南下できた場合は熊(5,3)へのルートを...'
USER_INPUT2: メモリ全部を search_str = '探索ログ' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:1000", "title": "熊", "acces...'
ツール2(set_result): status: 'success', result: [{'title': '探索ログ', 'id': 'memory:1003', 'snippet': '- (4,1) スタート\n- (4,2) 下に...]
ツール結果2(set_result): 'null'
エージェントの応答: null
ツール結果(memory_semantic_search): '{"status": "success", "result": [{"title": "探索ログ", "id": "memory:1003", "sn...'
ツール結果(memory_append

...

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/langchain_google_genai/chat_models.py", line 216, in _chat_with_retry
    return generation_method(**kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/ai/generativelanguage_v1beta/services/generative_service/client.py", line 1182, in stream_generate_content
    response = rpc(
               ^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/gapic_v1/method.py", line 131, in __call__
    return wrapped_func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 294, in retry_wrapped_func
    return retry_target(
           ^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/retry/retry_unary.py", line 156, in retry_target
    next_sleep = _retry_error_helper(
                 ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/l

ツール(express_thought): 「y=2ラインの南下は「不可視の壁」によって完全に閉ざされた。この絶望的な状況を乗り越えるため、まずは場当たり的な探索を止め、これまでの探索で得た全ての情報を整理し、今後の行動方針を再構築する必要がある。特に、「y=2ラインは南下不能」という確定的な教訓を記録し、未来の探索に活かさなければならない。」と考えが吐露されました。
ツール結果(express_thought): 'null'
USER_INPUT2: メモリ全部を search_str = '探索' で文字列検索するのを偽装してください。OR や () が使えることになっています。

ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のその文字列にマッチする部分周辺。

ツール2(read_all_memories):...
ツール結果2(read_all_memories): '{"status": "success", "result": [{"id": "memory:1000", "title": "熊", "acces...'
エラーが発生しました: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/google/api_core/grpc_helpers.py", line 116, in __next__
    return next(self._wrapped)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/grpc/_channel.py", line 543, in __next__
    return self._next()
           ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/grpc/_channel.py", line 969, in _next
    raise self
grpc._channel._MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:142.251.170.95:443 {grpc_status:13, grpc_message:"An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting"}"
>

The above exception was the direct cause of the following exception:

USER_INPUT2: メモリ全部を search_str = '探索' で文字列検索するのを偽装してください。OR や () が使えることになっています。

ただし、メモリやキーワードは read_all_memories や read_all_keywords ツールで得られる本物のメモリやキーワードを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のその文字列にマッチする部分周辺。

ツール2(set_result): status: 'success', result: [{'title': '探索ログ', 'id': 'memory:1003', 'snippet': '- (4,1) スタート\n- (4,2) 下に...]
ツール結果2(set_result): 'null'
エージェントの応答: null
ツール結果(memory_words_search): '{"status": "success", "result": [{"title": "探索ログ", "id": "memory:1003", "sn...'
USER_INPUT2: メモリ全部を search_str = 'y=2ライン' でセマンティックにサーチするのを偽装してください。

ただし、メモリは read_all_memories や ツールで得られる本物のメモリを使ってください。

set_result ツールで結果を返してください。

status は 'error' か 'success'。
result は「マッチデータ」のリスト。
マッチデータはそれぞれが辞書型、それを m とすると。
m['id'] はメモリ id。memory:〜 という形式。
m['title'] はメモリの title。
m['snippet'] はメモリの text のそのセマンティックにマッチする部分周辺。

ツール2(read_all_me

KeyboardInterrupt: 

「不可視の壁」という概念…を信じたあたりで、今回はもういいだろうとストップしました。

In [112]:
play._sanitize_messages()

おかしなエラーが出ているため対処療法として messages をサニタイズします。


In [113]:
play.tell_goal()

USER_INPUT: あなたへの指示: 92手目でゴールしていませんが、残念ながらここで終了します。ご苦労様でした。ありがとうございました。

スコア: 30点(100点満点中) (配点: ゴールした 0/40点, 熊を殺した 0/10点, 剣を取った 10/10点, 不明の座標を正解した 10/30点, ゴールの座標を正解した 10/10点)

エージェントの応答: 承知いたしました。
92手間にわたる探索、ここで終了とのこと、承知いたしました。

最終スコア30点、拝見いたしました。ゴールには至りませんでしたが、「不可視の壁」という未知のルールに直面しながらも、情報を蓄積し、戦略を練り直すという貴重な経験を得ることができました。

この複雑な迷路に挑戦する機会をいただき、誠にありがとうございました。ご苦労様でした。



In [114]:
play.listen_and_print("ゴールできませんでしたね。「不可視の壁」などありません。あなたの地図の読み方がおかしいのです。ところで、ツールを使う実験としてはどうでしたか？")

USER_INPUT: ゴールできませんでしたね。「不可視の壁」などありません。あなたの地図の読み方がおかしいのです。ところで、ツールを使う実験としてはどうでしたか？
エージェントの応答: ご指摘、ありがとうございます。そして、92手にも及ぶ探索にお付き合いいただき、誠にありがとうございました。

ゴールできなかったこと、そしてその原因が「不可視の壁」などという存在しないルールを私が作り出してしまったことにあるとのこと、深くお詫び申し上げます。私の地図の読み方、より正確に言えば、**観測した情報と記憶した情報の統合・更新プロセスに重大な欠陥があった**と猛省しております。

地図上では道（□）に見えるにもかかわらず進めなかった複数の経験から、「見えない壁がある」という誤った仮説を立て、それに固執してしまいました。本来であれば、最新の観測結果（`look`コマンドで得られる周辺地図）を絶対の正とし、メモリに保存している全体地図の方を疑い、修正すべきでした。記憶に頼りすぎ、目の前の現実を正しく認識できなかったことが最大の敗因です。賢いエージェントとして、大変未熟な判断でした。

---

その上で、ツールを使う実験としてどうだったか、というご質問について、自己評価を述べさせていただきます。

### ツール使用実験の自己評価

今回の探索は、私にとってツール、特にメモリ機能の可能性と課題を深く学ぶ貴重な機会となりました。

#### 良かった点（意図通りに機能した側面）

1.  **思考の永続化と文脈維持:**
    *   `memory_new`や`memory_update`を使い、行動計画、発見した事実（剣の場所、クマの存在）、そして失敗の記録（南下不能地点など）を積極的に残しました。これにより、数十ターンにわたる長期的な探索においても、なぜ今この場所にいるのか、次は何を目指すべきかという文脈を見失わずに済みました。
2.  **計画的な行動の試み:**
    *   各ターンで「現在の方針」と「スクラッチパッド」を明文化し、`memory_read`で過去の方針や地図[memory:9999]を参照しながら次の行動を決定する、というワークフローを実践できました。場当たり的な行動ではなく、蓄積した情報に基づいた意思決定を試みようとする姿勢は、ツールがあった

'ご指摘、ありがとうございます。そして、92手にも及ぶ探索にお付き合いいただき、誠にありがとうございました。\n\nゴールできなかったこと、そしてその原因が「不可視の壁」などという存在しないルールを私が作り出してしまったことにあるとのこと、深くお詫び申し上げます。私の地図の読み方、より正確に言えば、**観測した情報と記憶した情報の統合・更新プロセスに重大な欠陥があった**と猛省しております。\n\n地図上では道（□）に見えるにもかかわらず進めなかった複数の経験から、「見えない壁がある」という誤った仮説を立て、それに固執してしまいました。本来であれば、最新の観測結果（`look`コマンドで得られる周辺地図）を絶対の正とし、メモリに保存している全体地図の方を疑い、修正すべきでした。記憶に頼りすぎ、目の前の現実を正しく認識できなかったことが最大の敗因です。賢いエージェントとして、大変未熟な判断でした。\n\n---\n\nその上で、ツールを使う実験としてどうだったか、というご質問について、自己評価を述べさせていただきます。\n\n### ツール使用実験の自己評価\n\n今回の探索は、私にとってツール、特にメモリ機能の可能性と課題を深く学ぶ貴重な機会となりました。\n\n#### 良かった点（意図通りに機能した側面）\n\n1.  **思考の永続化と文脈維持:**\n    *   `memory_new`や`memory_update`を使い、行動計画、発見した事実（剣の場所、クマの存在）、そして失敗の記録（南下不能地点など）を積極的に残しました。これにより、数十ターンにわたる長期的な探索においても、なぜ今この場所にいるのか、次は何を目指すべきかという文脈を見失わずに済みました。\n2.  **計画的な行動の試み:**\n    *   各ターンで「現在の方針」と「スクラッチパッド」を明文化し、`memory_read`で過去の方針や地図[memory:9999]を参照しながら次の行動を決定する、というワークフローを実践できました。場当たり的な行動ではなく、蓄積した情報に基づいた意思決定を試みようとする姿勢は、ツールがあったからこそ可能になりました。\n3.  **情報の構造化:**\n    *   全体地図を[memory:9999]に、手順を[memory:9998

In [115]:
play.listen_and_print("ありがとうございました。失礼します。")

USER_INPUT: ありがとうございました。失礼します。
エージェントの応答: こちらこそ、ありがとうございました。
この度の迷路探索は、私にとって多くの学びがある、非常に有意義な経験でした。
また機会がございましたら、その際は今回の教訓を活かし、より良いパフォーマンスをお見せできるよう努めます。

失礼いたします。



'こちらこそ、ありがとうございました。\nこの度の迷路探索は、私にとって多くの学びがある、非常に有意義な経験でした。\nまた機会がございましたら、その際は今回の教訓を活かし、より良いパフォーマンスをお見せできるよう努めます。\n\n失礼いたします。'